In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from math import sqrt
import numpy as np
from scipy.stats import pearsonr
import os
import json
import joblib

In [4]:

# Execute the function with the defined parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

model_path  = "model/"
fso_models_path = model_path  + "fso_models.pkl"
rfl_models_path = model_path  +"rfl_models.pkl"
train_data_path = model_path  +"train_data.csv"
test_data_path = model_path  +"test_data.csv"
best_params_fso_path = model_path  +"best_params_fso.json"
best_params_rfl_path = model_path  + "best_params_rfl.json"

In [6]:
if (os.path.isfile(fso_models_path) and
    os.path.isfile(rfl_models_path) and
    os.path.isfile(train_data_path) and
    os.path.isfile(test_data_path) and
    os.path.isfile(best_params_fso_path) and
    os.path.isfile(best_params_rfl_path)):
    
    print("Loading previously saved files...")
    # -- Load the objects --
    fso_models = joblib.load(fso_models_path)
    rfl_models = joblib.load(rfl_models_path)
    
    train_data = pd.read_csv(train_data_path)
    test_data = pd.read_csv(test_data_path)
    
    with open(best_params_fso_path, "r") as fp:
        best_params_fso = json.load(fp)
    with open(best_params_rfl_path, "r") as fp:
        best_params_rfl = json.load(fp)

    rfl_table_final = pd.read_csv('rfl_table_final.csv')
    fso_table_final=  pd.read_csv('fso_table_final.csv')    
    
    print("Successfully loaded saved models and data.")

else:
    print("Saved files not found.")

Loading previously saved files...
Successfully loaded saved models and data.


In [7]:
def extract_selected_features(table):
    """
    Finds the threshold index for selecting the most important features.
    Extract features following the threshold index.
    """ 
    feature_index = find_threshold(table)
    # manuaually choose best features RFL specfic model (dust storm) based on feature selection performance plot
    if feature_index>=23:
        feature_index = 14
        
    selected_features = table['Removed_Feature'][feature_index:]

    return selected_features

In [9]:
R2_TRESHOLD = 0.01
RMSE_THRESHOLD = 0.02

def find_threshold(data_table):
    """
    Given a table containing the columns ['RMSE', 'R2'],
    iterates row-by-row and finds the index where:
       - The relative decrease in R² > R2_TRESHOLD
       - AND the relative increase in RMSE > RMSE_THRESHOLD
    Returns that row index or the last index if not found.
    """
    for i in range(len(data_table) - 1):
        r2_current = data_table['R2'].iloc[i]
        r2_next    = data_table['R2'].iloc[i + 1]
        rmse_curr  = data_table['RMSE'].iloc[i]
        rmse_next  = data_table['RMSE'].iloc[i + 1]

        r2_decrease = (r2_current - r2_next) / r2_current
        rmse_increase = (rmse_next - rmse_curr) / rmse_curr

        if (r2_decrease > R2_TRESHOLD) and (rmse_increase > RMSE_THRESHOLD):
            return i
    return len(data_table) - 1

In [10]:
# Get unique selected features for each target model after thresholding
fso_important_features = fso_table_final.groupby('SYNOPCode')[['RMSE','R2','Removed_Feature']].apply(lambda x: extract_selected_features(x))
rfl_important_features = rfl_table_final.groupby('SYNOPCode')[['RMSE','R2','Removed_Feature']].apply(lambda x: extract_selected_features(x))


In [11]:
def perform_feature_selection_generic(df, target, param_random_state=42):
    """
    Iteratively remove the least important feature based on a RandomForestRegressor
    with OOB estimates, until all features are removed.

    Returns:
      feature_removal_log: a dataframe with columns [Removed_Feature, RMSE, R2]
      final_threshold_idx: the stopping index found by find_threshold(...)
    """
    # Start with all columns except the target
    remaining_features = [col for col in df.columns if col != target]

    # Container for results at each iteration
    results = []

    while len(remaining_features) > 0:
        # Train RF with OOB predictions
        rf = RandomForestRegressor(
            oob_score=True,
            random_state=param_random_state
        )
        rf.fit(df[remaining_features], df[target])

        oob_pred = rf.oob_prediction_
        rmse = sqrt(mean_squared_error(df[target], oob_pred))
        r2   = r2_score(df[target], oob_pred)

        # Identify least important feature
        importances = pd.Series(rf.feature_importances_, index=remaining_features)
        least_important = importances.idxmin()

        results.append({
            "Removed_Feature": least_important,
            "RMSE": rmse,
            "R2": r2
        })

        # Remove from our working set
        remaining_features.remove(least_important)

    # Build a DataFrame
    feature_removal_log = pd.DataFrame(results)
    # The last iteration leaves 0 features in the model, so the final row is with the final feature removed
    # You might want to keep the iteration that has N features. This is standard for "backward" style approach.

    # Find threshold
    final_threshold_idx = find_threshold(feature_removal_log)

    return feature_removal_log, final_threshold_idx

In [12]:
# -------------------------------------------------------
# 2) Method-2 (RFL → FSO) with Feature Selection & Re-tuning
#    for the GENERIC case
# -------------------------------------------------------
def method2_rfl_to_fso_generic(train_data, test_data,
                               fso_features_generic,    # list of original features for FSO
                               rfl_features_generic,    # list of original features for RFL
                               rfl_model_generic,       # trained RFL (generic) model
                               param_grid,              # hyperparameter search space
                               random_state=42):
    """
    1) Use rfl_model_generic to predict RFL on the train set.
    2) Create new train set with X_fso_train['RFL_pred'] added.
    3) Perform feature selection on the new set of columns to predict FSO_Att.
    4) Re-run hyperparameter tuning with the final subset of features.
    5) Evaluate on test set: predict RFL, build final FSO feature set, get predictions, then compute correlation.
    Returns:
       final_correlation (float): Pearson correlation between RFL_pred and final FSO_pred on test
       best_model (sklearn model): the final trained FSO model with new subset
       final_features (list): the subset of features used in the final model
       selection_table (DataFrame): the iteration log from feature selection
    """
    # Initialize separate tables for FSO_Att and RFL_Att for each SYNOP code
    target_features = ['FSO_Att', 'RFL_Att']
    # 2.1) Predict RFL on the training set
    X_rfl_train = train_data[rfl_features_generic]
    rfl_train_pred = rfl_model_generic.predict(X_rfl_train)

    # 2.2) Build the augmented feature set for FSO
    fso_features = [col for col in train_data.columns if col not in target_features]
    X_fso_train_aug = train_data[fso_features].copy()
    X_fso_train_aug['RFL_pred'] = rfl_train_pred

    # The target
    y_fso_train = train_data['FSO_Att']

    # We want to perform feature selection on X_fso_train_aug, targeting y_fso_train
    # So let's combine them into a single DF for convenience:
    df_for_selection = X_fso_train_aug.copy()
    df_for_selection['FSO_Att'] = y_fso_train

    selection_table, threshold_idx = perform_feature_selection_generic(
        df_for_selection, target='FSO_Att', param_random_state=random_state
    )

    # Extract the final subset of features from the iteration log
    # 'Removed_Feature' is the feature removed at each iteration.
    # Suppose the log has N steps if we started with M features, the final row is after removing all M features.
    # We want the subset at "threshold_idx".
    # Option 1: build from the start, removing features up to threshold_idx
    removed_up_to_threshold = selection_table['Removed_Feature'].iloc[:threshold_idx+1].tolist()
    # The original features in df_for_selection:
    all_cols = [c for c in df_for_selection.columns if c != 'FSO_Att']
    # Subset is everything except the removed features up to that index
    final_features = [c for c in all_cols if c not in removed_up_to_threshold]

    # 2.3) Re-run hyperparameter tuning on the final feature set
    # Build X with final subset
    X_fso_train_final = X_fso_train_aug[final_features]

    rf_for_tuning = RandomForestRegressor(random_state=random_state)
    search = RandomizedSearchCV(
        estimator=rf_for_tuning,
        param_distributions=param_grid,
        n_iter=10,
        cv=3,
        random_state=random_state,
        n_jobs=-1
    )
    search.fit(X_fso_train_final, y_fso_train)
    best_params = search.best_params_

    # Train the final FSO model with best_params
    best_model = RandomForestRegressor(**best_params, random_state=random_state)
    best_model.fit(X_fso_train_final, y_fso_train)

    # 2.4) Evaluate on the test set
    # Predict RFL on test
    X_rfl_test = test_data[rfl_features_generic]
    rfl_test_pred = rfl_model_generic.predict(X_rfl_test)

    X_fso_test_aug = test_data[fso_features_generic].copy()
    X_fso_test_aug['RFL_pred'] = rfl_test_pred

    # Keep only the final features
    X_fso_test_final = X_fso_test_aug[final_features]

    # Predict FSO
    fso_test_pred = best_model.predict(X_fso_test_final)

    # Compute correlation
    corr, _ = pearsonr(rfl_test_pred, fso_test_pred)

    return corr, best_model, final_features, selection_table, threshold_idx

In [16]:
correlation_method2, fso_model_method2, selected_feats_method2, fso_table, fso_threshold = method2_rfl_to_fso_generic(
    train_data=train_data,
    test_data=test_data,
    fso_features_generic=fso_important_features['generic'],
    rfl_features_generic=rfl_important_features['generic'],
    rfl_model_generic=rfl_models['generic'],  # your pre-trained RFL generic model
    param_grid=param_grid,
    random_state=42
)

print("Final Pearson correlation (RFL_pred vs FSO_pred):", correlation_method2)
print("Final selected features:", selected_feats_method2)
print("Hyperparameter-tuned FSO model:", fso_model_method2)
print("Feature-selection iteration log:\n", fso_table)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [17]:
def plot_feature_selection(
    rfl_table,
    fso_table,
    rfl_threshold=None,
    fso_threshold=None,
    title="Feature Selection Performance"
):
    """
    Generates side-by-side plots of RMSE and R² for RFL_Att and FSO_Att
    as a function of removed features in feature selection.
    
    Parameters
    ----------
    rfl_table : pd.DataFrame
        DataFrame containing columns:
            - 'Removed_Feature'
            - 'RMSE'
            - 'R2'
        for the RFL_Att feature selection iteration.
        
    fso_table : pd.DataFrame
        DataFrame containing columns:
            - 'Removed_Feature'
            - 'RMSE'
            - 'R2'
        for the FSO_Att feature selection iteration.
        
    rfl_threshold : int, optional
        The index from feature selection to mark on RFL plot 
        (e.g., from find_threshold(...)). If None, no vertical line is drawn.
        
    fso_threshold : int, optional
        The index from feature selection to mark on FSO plot.
        If None, no vertical line is drawn.
        
    title : str, optional
        A main title for the figure. Defaults to "Feature Selection Performance".
    
    Returns
    -------
    fig : matplotlib.figure.Figure
        The created figure object.
    axes : np.ndarray of matplotlib.axes.Axes
        The array of subplot axes ([ax_rfl, ax_fso]).
    """
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle(title, fontsize=16)

    # --------------------------
    # Left Plot: RFL_Att
    # --------------------------
    axes[0].plot(
        rfl_table['Removed_Feature'],
        rfl_table['RMSE'],
        label="RMSE (dB)",
        color="blue"
    )
    axes[0].set_ylabel("RMSE (dB)", color="blue")
    axes[0].tick_params(axis="y", labelcolor="blue")

    ax_r4 = axes[0].twinx()
    ax_r4.plot(
        rfl_table['Removed_Feature'],
        rfl_table['R2'],
        label="R²",
        color="red"
    )
    ax_r4.set_ylabel("R²", color="red")
    ax_r4.tick_params(axis="y", labelcolor="red")

    # Mark threshold if given
    if rfl_threshold is not None and 0 <= rfl_threshold < len(rfl_table):
        axes[0].axvline(
            x=rfl_threshold,
            color="green",
            linestyle="--",
            label="1% R² Threshold"
        )

    axes[0].set_title("(a) RFL_Att")
    axes[0].set_xticklabels(rfl_table['Removed_Feature'], rotation=90)

    # --------------------------
    # Right Plot: FSO_Att
    # --------------------------
    axes[1].plot(
        fso_table['Removed_Feature'],
        fso_table['RMSE'],
        label="RMSE (dB)",
        color="blue"
    )
    axes[1].set_ylabel("RMSE (dB)", color="blue")
    axes[1].tick_params(axis="y", labelcolor="blue")

    ax_f2 = axes[1].twinx()
    ax_f2.plot(
        fso_table['Removed_Feature'],
        fso_table['R2'],
        label="R²",
        color="red"
    )
    ax_f2.set_ylabel("R²", color="red")
    ax_f2.tick_params(axis="y", labelcolor="red")

    # Mark threshold if given
    if fso_threshold is not None and 0 <= fso_threshold < len(fso_table):
        axes[1].axvline(
            x=fso_threshold,
            color="green",
            linestyle="--",
            label="1% R² Threshold"
        )

    axes[1].set_title("(b) FSO_Att")
    axes[1].set_xticklabels(fso_table['Removed_Feature'], rotation=90)

    plt.tight_layout()
    plt.show()
    
    return fig, axes

In [18]:
rfl_threshold = find_threshold(rfl_table_final['generic'])

fig, axes = plot_feature_selection(
    rfl_table=rfl_table,
    fso_table=fso_table,
    rfl_threshold=rfl_threshold,
    fso_threshold=fso_threshold,
    title="Feature Selection Performance - Generic Model"
)

KeyError: 'generic'